Pre All that:
Convert Cell names to Column names in the formulas 
(Think about how we can keep all formulas), get all formulas, pass them through an llm and make it remove the redundant ones 


Take each table, pass through an LLM:

Ask it if its got columns around "SKU, location/warehouse, quantity" - for inventory planning (boolean)
Ask it to mark other tables if they're related to inventory planning (boolean)

For all 1s:
    Pass table and metadata through llm and identify specific data ranges by their column labels and formulas
    

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook

# Load Excel Workbook
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
workbook = load_workbook(file_path, data_only=False, read_only=True)
sheet_names = [sheet_name for sheet_name in workbook.sheetnames if not workbook[sheet_name].sheet_state == 'hidden']

# sheet_names = workbook.sheetnames

# Function to extract the first 50 rows and all columns from a sheet
def extract_sheet_data(sheet_name):
    if sheet_name not in sheet_names:
        print(f"Sheet '{sheet_name}' does not exist in the workbook.")
    else:
        print(f"Processing sheet: {sheet_name}")
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    rows_data = []
    for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols, values_only=False):
        rows_data.append([cell for cell in row])
    return rows_data

# Tool to fetch sheet data
def get_sheet_data(sheet_name: str) -> str:
    rows = extract_sheet_data(sheet_name)
    data = []
    for row in rows:
        data.append([cell.value if cell else "" for cell in row])
    return data

# Tool to analyze sheet data and generate Python code
def analyze_and_generate_code(sheet_name: str) -> str:
    data = get_sheet_data(sheet_name)
    # Convert data into a human-readable text format for the LLM
    formatted_data = "\n".join([", ".join(map(str, row)) for row in data])
    llm_input = f"""
    You are a Python code generator and table extraction expert.
    Below is the first 50 rows of data from the sheet '{sheet_name}':

    {formatted_data}

    Your tasks:
    1. Detect all tables in this data.
    2. Extract table column names.
    3. Identify unique formulas used in each column and the column dependencies in these formulas.
    4. Generate Python code to extract the tables, their column names, and formulas programmatically.
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = llm.invoke(llm_input)
    return response

# Define Tools for the LangChain Agent
tools = [
    Tool(
        name="ExtractSheetData",
        func=lambda sheet_name: get_sheet_data(sheet_name),
        description="Extracts the first 50 rows and all columns from a given sheet."
    ),
    Tool(
        name="AnalyzeSheetAndGenerateCode",
        func=lambda sheet_name: analyze_and_generate_code(sheet_name),
        description="Analyzes a sheet and generates Python code to extract tables, column names, and formulas."
    ),
]

# Initialize LangChain Agent
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Use the agent to analyze all sheets
for sheet_name in sheet_names:
    print(f"Analyzing sheet: {sheet_name}")
    response = agent.run(f"Analyze the sheet {sheet_name} and generate code.")
    print(response)
    break


Analyzing sheet: INVENTORY SNAPSHOT


> Entering new AgentExecutor chain...
I need to analyze the sheet named "INVENTORY SNAPSHOT" to generate Python code that will help extract tables, column names, and formulas from it. 
Action: AnalyzeSheetAndGenerateCode
Action Input: "INVENTORY SNAPSHOT"Processing sheet: INVENTORY SNAPSHOT

Observation: content='To accomplish the tasks you\'ve outlined, we will follow a structured approach. Below is a breakdown of the steps involved, along with the corresponding Python code to extract the tables, their column names, and formulas programmatically.\n\n### Step 1: Detect Tables in the Data\nFrom the provided data, we can identify that there are multiple tables based on the presence of headers and structured data. The first table appears to start from the row containing "SKU Description" and continues until the end of the relevant data.\n\n### Step 2: Extract Table Column Names\nThe column names can be extracted from the row that contains the headers.

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook
import pandas as pd

# Load Excel Workbook
file_path = "/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx"  # Replace with your file path
workbook = load_workbook(file_path, data_only=True, read_only=True)
sheet_names = workbook.sheetnames

# Function to extract a 50xN chunk from a sheet
def extract_sheet_chunk(sheet_name):
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    if sheet_name in sheet_names:
        print(f"Sheet {sheet_name.strip()} exists")
    else:
        print(f"Sheet {sheet_name.strip()} doesn't exist")
        return False
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    data = [[cell.value for cell in row] for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols)]
    return data

# Tool: Fetch the sheet chunk
def get_sheet_chunk(sheet_name: str) -> str:
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    chunk = extract_sheet_chunk(sheet_name)
    formatted_chunk = "\n".join([", ".join(map(str, row)) for row in chunk])
    return formatted_chunk

# Tool: Analyze and generate code for extracting tables
def analyze_and_generate_code(sheet_name: str) -> str:
    chunk = get_sheet_chunk(sheet_name)
    llm_input = f"""
    You are a Python code generator for table extraction.
    Below is a 50xN chunk of data from the sheet '{sheet_name}':

    {chunk}

    Tasks:
    1. Analyze the data to identify all tables present.
    2. Generate Python code to extract each table programmatically.
    3. The code should extract each table into a pandas DataFrame and save it as a separate CSV file.
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = llm.invoke(llm_input)
    return response

# Tool Definitions for LangChain
tools = [
    Tool(
        name="GetSheetChunk",
        func=lambda sheet_name: get_sheet_chunk(sheet_name),
        description="Fetches a 50xN chunk of data from the specified sheet."
    ),
    Tool(
        name="AnalyzeAndGenerateCode",
        func=lambda sheet_name: analyze_and_generate_code(sheet_name),
        description="Analyzes a sheet chunk and generates Python code to extract and save tables."
    ),
    Tool(
        name="PythonREPL",
        func=PythonREPLTool().run,
        description="Executes Python code in a REPL environment."
    ),
]

# Initialize LangChain Agent
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Run the agent on a specific sheet
sheet_to_analyze = "INVENTORY SNAPSHOT"  # Replace with your target sheet
try:
    print(f"Analyzing sheet: {sheet_to_analyze}")
    response = agent.run(f"Analyze the sheet {sheet_to_analyze} and generate code.")
    print(response)

    # Execute the generated code
    execution_response = agent.run(f"Execute the following Python code: {response}")
    print(execution_response)

except Exception as e:
    print(f"Error: {e}")


Analyzing sheet: INVENTORY SNAPSHOT


> Entering new AgentExecutor chain...
I need to analyze the sheet named "INVENTORY SNAPSHOT" to understand its structure and content before generating the appropriate Python code to extract and save the tables. 

Action: GetSheetChunk  
Action Input: "INVENTORY SNAPSHOT"  
Sheet INVENTORY SNAPSHOT exists

Observation: None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
None, None, 2025-01-10 00:00:00, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 45 = antsy, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None,

In [37]:
import traceback
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook

# Load Excel Workbook
file_path = "/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx"  # Replace with actual file path
workbook = load_workbook(file_path, data_only=True, read_only=True)
sheet_names = workbook.sheetnames

# Function to extract a 50xN chunk from a sheet
def extract_sheet_chunk(sheet_name):
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    if sheet_name not in sheet_names:
        log_message = f"Sheet '{sheet_name}' doesn't exist. Available sheets: {sheet_names}"
        print(log_message)
        return log_message
    print(f"Fetching 50xN chunk from sheet: {sheet_name}")
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    data = [[cell.value for cell in row] for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols)]
    formatted_chunk = "\n".join([", ".join(map(str, row)) for row in data])
    print(f"Fetched chunk from sheet '{sheet_name}':\n{formatted_chunk[:500]}...")  # Show first 500 chars for brevity
    return formatted_chunk

# Function to analyze a chunk and generate code
import re

def analyze_and_generate_code(sheet_name: str, chunk: str) -> str:
    llm_input = f"""
    You are a Python code generator for table extraction.
    Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

    {chunk}

    Tasks:
    1. Analyze the data to identify all tables present.
    2. Generate Python code that reads data from the provided Excel file ('{file_path}') and the specified sheet ('{sheet_name}').
    3. Extract each identified table and save it as a separate CSV file.
    """
    print(f"Sending data to LLM for analysis and code generation...")
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    try:
        response = llm.invoke(llm_input)
        generated_response = response.content  # Extract string content
        print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

        # Extract Python code block using regex
        match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
        if match:
            generated_code = match.group(1).strip()  # Extract the Python code
            print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
            return generated_code
        else:
            print("No valid Python code block found in the LLM response.")
            return "Error: No valid Python code block found."
    except Exception as e:
        print(f"Error during code generation: {e}")
        return f"Error during code generation: {e}"

# Function to execute generated Python code
def execute_code(code: str) -> str:
    try:
        print(f"Executing the following code:\n{code[:500]}...")  # Log the extracted code
        repl_tool = PythonREPLTool()
        result = repl_tool.run(code)
        print("Execution result:")
        print(result)
        return result
    except Exception as e:
        error_message = f"Execution Error: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        return error_message

# Main execution process
def run_analysis(sheet_name):
    print(f"Starting analysis for sheet: {sheet_name}")
    
    # Step 1: Fetch sheet chunk
    chunk = extract_sheet_chunk(sheet_name)
    if "doesn't exist" in chunk:
        print(f"Error: {chunk}")
        return chunk

    # Step 2: Analyze and generate code
    print("Analyzing the data and generating Python code...")
    code = analyze_and_generate_code(sheet_name, chunk)
    if "Error during code generation" in code:
        return code

    # Step 3: Attempt to execute code with retries
    retry = 10
    while retry > 0:
        result = execute_code(code)
        if "Execution Error" not in result:
            print("Code executed successfully!")
            return result  # Successful execution

        # Provide feedback and request fixes
        error_message = f"\n\nError Message: {result}"
        print(f"Retrying... ({10 - retry + 1}/10)")
        print(f"Requesting LLM to fix the code. Error: {error_message}")
        code = analyze_and_generate_code(sheet_name, chunk + error_message)
        retry -= 1

    print("Exceeded maximum retries. Could not execute code successfully.")
    return "Failed after multiple retries."

# Run the workflow
sheet_to_analyze = "Current Inventory"  # Replace with your target sheet name
print(run_analysis(sheet_to_analyze))


Starting analysis for sheet: Current Inventory
Fetching 50xN chunk from sheet: Current Inventory
Fetched chunk from sheet 'Current Inventory':
INVENTORY BY LOCATION, None, None, None, None, INVENTORY TOTALS -- EXCLUDING EGS, None, None, None
None, None, None, None, None, None, None, None, None
Date, 2025-01-05 00:00:00, None, None, None, Date, 2024-11-14 00:00:00, None, None
None, None, None, None, None, None, None, None
Location, Description, Code, Case Quantity per Tracker AS OF DATE, None, Description, Code, Case Quantity per Tracker AS OF DATE
Comarco Products, CP Big Mozz Sticks, 20005-1, -184, None, CP Big Mozz Sticks, 20005-1, 1...
Analyzing the data and generating Python code...
Sending data to LLM for analysis and code generation...
Full LLM response:
Here is the Python code that uses pandas to read the Excel file, identify the tables, and save them as separate CSV files:

```python
import pandas as pd

# Read the Excel file
df = pd.read_excel('/Users/ajay/Documents/Atomic/inv

In [ ]:
import os 
import pandas as pd 
for filename in os.listdir(os.getcwd()):
    if 'table_' in filename:
        print(filename)
        display(pd.read_csv(filename))

In [52]:
import traceback
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook

# Load Excel Workbook
file_path = "/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx"  # Replace with actual file path
workbook = load_workbook(file_path, data_only=True, read_only=True)
sheet_names = workbook.sheetnames

# Function to extract a 50xN chunk from a sheet
def extract_sheet_chunk(sheet_name):
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    if sheet_name not in sheet_names:
        log_message = f"Sheet '{sheet_name}' doesn't exist. Available sheets: {sheet_names}"
        print(log_message)
        return log_message
    print(f"Fetching 50xN chunk from sheet: {sheet_name}")
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    data = [[cell.value for cell in row] for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols)]
    formatted_chunk = "\n".join([", ".join(map(str, row)) for row in data])
    print(f"Fetched chunk from sheet '{sheet_name}':\n{formatted_chunk[:500]}...")  # Show first 500 chars for brevity
    return formatted_chunk

# Function to analyze a chunk and generate code
import re

# def analyze_and_generate_code(sheet_name: str, chunk: str) -> str:
#     llm_input = f"""
#     You are a Python code generator for table extraction.
#     Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

#     {chunk}

#     Tasks:
#     1. Analyze the data to identify all tables present.
#     2. Generate Python code that reads data from the provided Excel file ('{file_path}') and the specified sheet ('{sheet_name}').
#     3. Create a new folder named '{sheet_name}' in the current working directory.
#     4. Extract each identified table and save it as a separate CSV file in the folder '{sheet_name}'.
#     """
#     print(f"Sending data to LLM for analysis and code generation...")
#     llm = ChatOpenAI(model="gpt-4", temperature=0)
#     try:
#         response = llm.invoke(llm_input)
#         generated_response = response.content  # Extract string content
#         print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

#         # Extract Python code block using regex
#         match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
#         if match:
#             generated_code = match.group(1).strip()  # Extract the Python code
#             print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
#             return generated_code
#         else:
#             print("No valid Python code block found in the LLM response.")
#             return "Error: No valid Python code block found."
#     except Exception as e:
#         print(f"Error during code generation: {e}")
#         return f"Error during code generation: {e}"

# best working - Only gets all the csvs. Doesn't get formulas yet
def analyze_and_generate_code(sheet_name: str, chunk: str) -> str:
    llm_input = f"""
    You are a Python code generator for table extraction.
    Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

    {chunk}

    Tasks:
    1. Analyze the data to identify all tables present in the chunk. Tables are defined as contiguous blocks of data separated by blank rows or rows with no data.
    2. Generate Python code that:
       - Reads data from the provided Excel file ('{file_path}') and specified sheet ('{sheet_name}').
       - Creates a new folder named '{sheet_name}'.
       - Extracts each identified table based on the above definition and saves it as a separate CSV file in the folder.
    3. Do not make any assumptions about the meaning or functionality of specific columns unless explicitly mentioned.
    """
    print(f"Sending data to LLM for analysis and code generation...")
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    try:
        response = llm.invoke(llm_input)
        generated_response = response.content  # Extract string content
        print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

        # Extract Python code block using regex
        match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
        if match:
            generated_code = match.group(1).strip()  # Extract the Python code
            print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
            return generated_code
        else:
            print("No valid Python code block found in the LLM response.")
            return "Error: No valid Python code block found."
    except Exception as e:
        print(f"Error during code generation: {e}")
        return f"Error during code generation: {e}"





# Function to execute generated Python code
def execute_code(code: str) -> str:
    try:
        print(f"Executing the following code:\n{code}...")  # Log the extracted code
        repl_tool = PythonREPLTool()
        result = repl_tool.run(code)
        print("Execution result:")
        print(result)
        return result
    except Exception as e:
        error_message = f"Execution Error: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        return error_message

# Main execution process
def run_analysis(sheet_name):
    print(f"Starting analysis for sheet: {sheet_name}")
    
    # Step 1: Fetch sheet chunk
    chunk = extract_sheet_chunk(sheet_name)
    if "doesn't exist" in chunk:
        print(f"Error: {chunk}")
        return chunk

    # Step 2: Analyze and generate code
    print("Analyzing the data and generating Python code...")
    code = analyze_and_generate_code(sheet_name, chunk)
    if "Error during code generation" in code:
        return code

    # Step 3: Attempt to execute code with retries
    retry = 10
    while retry > 0:
        result = execute_code(code)
        if "Execution Error" not in result:
            print("Code executed successfully!")
            return result  # Successful execution

        # Provide feedback and request fixes
        error_message = f"\n\nError Message: {result}"
        print(f"Retrying... ({10 - retry + 1}/10)")
        print(f"Requesting LLM to fix the code. Error: {error_message}")
        code = analyze_and_generate_code(sheet_name, chunk + error_message)
        retry -= 1

    print("Exceeded maximum retries. Could not execute code successfully.")
    return "Failed after multiple retries."

# Run the workflow
sheet_to_analyze = "True Up Template"  # Replace with your target sheet name
print(run_analysis(sheet_to_analyze))


Starting analysis for sheet: True Up Template
Fetching 50xN chunk from sheet: True Up Template
Fetched chunk from sheet 'True Up Template':
Date, 2024-10-03 00:00:00, None, None, None, None, None, None, None, None, None, None, None, None, None, None
None, None, None, None, None, None, None, None, None, None, None, None, None, COUNTS FROM WAREHOUSES, None, None, None, None, None, None, None, None, None, None, None, None
Location, Description, Code, Case Quantity per Tracker AS OF DATE, Actual Quantity AS OF DATE, Difference, None, None, None, None, None, Location, Description, Jan 11th 2024, Jan 21st 2024, 2024-02-01 00:00:00, Mid-Fe...
Analyzing the data and generating Python code...
Sending data to LLM for analysis and code generation...
Full LLM response:
Here is the Python code to perform the tasks:

```python
import openpyxl
from openpyxl.utils import get_column_letter, column_index_from_string

def extract_formulas(file_path, sheet_name):
    # Load workbook with data_only=False t

/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/packaging/relationship.py:118: UserWarning: xl/pivotCache/_rels/pivotCacheDefinition1.xml.rels contains invalid dependency definitions
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/packaging/relationship.py:118: UserWarning: xl/pivotCache/_rels/pivotCacheDefinition2.xml.rels contains invalid dependency definitions
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/packaging/relationship.py:118: UserWarning: xl/pivotCache/_rels/pivotCacheDefinition3.xml.rels contains invalid dependency definitions
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/packaging/relationship.py:118: UserWarning: xl/pivotCache/_rels/pivotCacheDefinition4.xml.rels contains invalid dependency definitions
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/packaging/relationship.py:118: U

Execution result:
{'Date': None, datetime.datetime(2024, 10, 3, 0, 0): None, None: None}

Code executed successfully!
{'Date': None, datetime.datetime(2024, 10, 3, 0, 0): None, None: None}



In [ ]:
import os 
import pandas as pd 
for filename in os.listdir(sheet_to_analyze):
    if '.csv' in filename:
        print(filename)
        display(pd.read_csv(os.path.join(os.getcwd(),sheet_to_analyze,filename)))

Check if a table is related to inventory planning

In [98]:
import os
import pandas as pd
from pydantic import BaseModel, Field, ValidationError
from langchain.chat_models import ChatOpenAI
import json


# Define the structured output schema
class InventorySheetAnalysis(BaseModel):
    is_inventory_planning: int = Field(
        ..., description="1 if the sheet is for inventory planning, 0 otherwise"
    )
    details: dict = Field(
        ...,
        description=(
            "A dictionary mapping columns to roles: SKU, Location/Warehouse, Quantity, "
            "Total Inventory, Current Inventory, Sales Forecast. Set to None if not found."
        ),
    )
    description: str = Field(
        ..., description="Explanation of why this sheet is for inventory planning."
    )


# Function to clean the LLM response
def clean_response(response: str) -> str:
    """
    Cleans the LLM response by removing Markdown fences and extra whitespace.

    Args:
        response (str): The raw response from the LLM.

    Returns:
        str: The cleaned JSON string.
    """
    try:
        # Remove Markdown fences (e.g., ```json ... ```)
        if response.startswith("```json"):
            response = response[7:]  # Remove ```json
        if response.endswith("```"):
            response = response[:-3]  # Remove ```
        return response.strip()  # Strip any extra whitespace
    except Exception as e:
        print(f"Error cleaning response: {e}")
        return response  # Return as-is if cleaning fails


# Function to analyze a sheet using GPT-4o-mini
def analyze_sheet(dataframe, llm, retries=10):
    # Convert the first 5 rows of the dataframe to Markdown
    markdown_preview = dataframe.head(5).to_markdown()

    # Define the LLM prompt
    prompt = f"""
    You are a data analysis expert. Below is a preview of a sheet:

    {markdown_preview}

    Task:
    1. Determine if this sheet is used for inventory planning (1 for Yes, 0 for No).
    2. If Yes, provide:
       - 'details': A dictionary mapping column names to these roles: SKU, Location/Warehouse, Quantity, Total Inventory, Current Inventory, Sales Forecast. If no related column exists, set the value to None.
       - 'description': A brief explanation explaining why this sheet is for inventory planning.

    Respond strictly in the following JSON format:
    {{
        "is_inventory_planning": 1,
        "details": {{
            "SKU": "Column Name or None",
            "Location/Warehouse": "Column Name or None",
            "Quantity": "Column Name or None",
            "Total Inventory": "Column Name or None",
            "Current Inventory": "Column Name or None",
            "Sales Forecast": "Column Name or None"
        }},
        "description": "Brief explanation"
    }}
    """

    # Retry loop
    while retries > 0:
        try:
            # Send the prompt to the LLM
            response = llm.predict(prompt)

            # Clean the response
            cleaned_response = clean_response(response)
            print(f"Cleaned Response:\n{cleaned_response}")

            # Validate the cleaned response with Pydantic using model_validate_json
            structured_output = InventorySheetAnalysis.model_validate_json(cleaned_response)
            return structured_output  # Return the validated output if successful

        except ValidationError as ve:
            print(f"Validation Error: {ve}")
            retries -= 1
            print(f"Retries left: {retries}")
            if retries == 0:
                print("Max retries reached. Skipping this sheet.")
                return None
        except Exception as e:
            print(f"Unexpected Error: {e}")
            return None


# Function to process all CSV files in a base folder
def analyze_csv_files(base_folder, llm):
    results = {}

    for folder in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".csv"):
                    file_path = os.path.join(folder_path, file)
                    print(f"Processing file: {file_path}")

                    # Load the CSV file
                    df = pd.read_csv(file_path)

                    # Analyze the sheet
                    response = analyze_sheet(dataframe=df, llm=llm)
                    if response:
                        results[file] = response.dict()  # Save structured output
                    else:
                        print(f"Failed to analyze file: {file}")

    return results


# Main Execution
# Initialize the GPT-4o-mini model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Define the base folder path for analysis
base_folder = "/Users/ajay/Documents/Atomic/inventory_analysis_2"  # Replace with your folder path
analysis_results = analyze_csv_files(base_folder, llm)

# Print Results
for file, result in analysis_results.items():
    print(f"\nFile: {file}")
    print(f"Inventory Planning: {'Yes' if result['is_inventory_planning'] == 1 else 'No'}")
    if result['is_inventory_planning'] == 1:
        print(f"Details: {result['details']}")
        print(f"Description: {result['description']}")


Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/True Up Template/table_2.csv
Cleaned Response:
{
    "is_inventory_planning": 0,
    "details": {
        "SKU": null,
        "Location/Warehouse": null,
        "Quantity": null,
        "Total Inventory": null,
        "Current Inventory": null,
        "Sales Forecast": null
    },
    "description": "This sheet does not contain any identifiable columns related to inventory planning, such as SKU, location, or inventory quantities."
}
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/True Up Template/table_3.csv


/var/folders/2j/n6fpg8f54xb5ld5v6bg1wgl80000gn/T/ipykernel_89927/145246337.py:124: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  results[file] = response.dict()  # Save structured output


Cleaned Response:
{
    "is_inventory_planning": 1,
    "details": {
        "SKU": "Product Description",
        "Location/Warehouse": "Location",
        "Quantity": "Cases",
        "Total Inventory": "Unnamed: 12",
        "Current Inventory": "Unnamed: 13",
        "Sales Forecast": "Unnamed: 14"
    },
    "description": "This sheet contains data related to product descriptions, quantities, and locations, which are essential for inventory planning and management."
}
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/True Up Template/table_1.csv


/var/folders/2j/n6fpg8f54xb5ld5v6bg1wgl80000gn/T/ipykernel_89927/145246337.py:124: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  results[file] = response.dict()  # Save structured output


Cleaned Response:
{
    "is_inventory_planning": 1,
    "details": {
        "SKU": "Unnamed: 3",
        "Location/Warehouse": "Date",
        "Quantity": "Unnamed: 4",
        "Total Inventory": "Unnamed: 6",
        "Current Inventory": "Unnamed: 5",
        "Sales Forecast": "Unnamed: 10"
    },
    "description": "This sheet is used for inventory planning as it contains information about product SKUs, their quantities, and inventory levels, which are essential for managing stock and forecasting sales."
}
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_2.csv


...

Cleaned Response:
{
    "is_inventory_planning": 0,
    "details": {
        "SKU": null,
        "Location/Warehouse": null,
        "Quantity": null,
        "Total Inventory": null,
        "Current Inventory": null,
        "Sales Forecast": null
    },
    "description": "The sheet does not contain any identifiable columns related to inventory planning."
}
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Delivery by Month/table_120.csv


/var/folders/2j/n6fpg8f54xb5ld5v6bg1wgl80000gn/T/ipykernel_89927/145246337.py:124: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  results[file] = response.dict()  # Save structured output


Cleaned Response:
{
    "is_inventory_planning": 0,
    "details": {
        "SKU": null,
        "Location/Warehouse": null,
        "Quantity": null,
        "Total Inventory": null,
        "Current Inventory": null,
        "Sales Forecast": null
    },
    "description": "This sheet does not contain any identifiable columns related to inventory planning."
}
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Delivery by Month/table_90.csv


/var/folders/2j/n6fpg8f54xb5ld5v6bg1wgl80000gn/T/ipykernel_89927/145246337.py:124: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  results[file] = response.dict()  # Save structured output


KeyboardInterrupt: 

Check if a table contains information related to Inventory - 0.02$

In [101]:
import os
import pandas as pd
from pydantic import BaseModel, Field, ValidationError
from langchain.chat_models import ChatOpenAI
import json


# Define the structured output schema
class InventorySheetAnalysis(BaseModel):
    is_inventory_planning: int = Field(
        ..., description="1 if the sheet is for inventory planning, 0 otherwise"
    )
    details: dict = Field(
        ...,
        description=(
            "A dictionary mapping columns to roles: SKU, Location/Warehouse, Quantity, "
            "Total Inventory, Current Inventory, Sales Forecast. Set to None if not found."
        ),
    )
    description: str = Field(
        ..., description="Explanation of why this sheet is for inventory planning."
    )


# Function to clean the LLM response
def clean_response(response: str) -> str:
    if response.startswith("```json"):
        response = response[7:]  # Remove ```json
    if response.endswith("```"):
        response = response[:-3]  # Remove ```
    return response.strip()  # Strip any extra whitespace


# Function to analyze a sheet using GPT-4o-mini
def analyze_sheet(dataframe, llm, retries=10):
    markdown_preview = dataframe.head(5).to_markdown()

    prompt = f"""
    You are a data analysis expert. Below is a preview of a sheet:

    {markdown_preview}

    Task:
    1. Determine if this sheet is used for inventory planning (1 for Yes, 0 for No).
    2. If Yes, provide:
       - 'details': A dictionary mapping column names to these roles: SKU, Location/Warehouse, Quantity, Total Inventory, Current Inventory, Sales Forecast. If no related column exists, set the value to None.
       - 'description': A brief explanation explaining why this sheet is for inventory planning.

    Respond strictly in the following JSON format:
    {{
        "is_inventory_planning": 1,
        "details": {{
            "SKU": "Column Name or None",
            "Location/Warehouse": "Column Name or None",
            "Quantity": "Column Name or None",
            "Total Inventory": "Column Name or None",
            "Current Inventory": "Column Name or None",
            "Sales Forecast": "Column Name or None"
        }},
        "description": "Brief explanation"
    }}
    """

    while retries > 0:
        try:
            response = llm.predict(prompt)
            cleaned_response = clean_response(response)
            structured_output = InventorySheetAnalysis.model_validate_json(cleaned_response)
            return structured_output
        except ValidationError as ve:
            print(f"Validation Error: {ve}")
            retries -= 1
            print(f"Retries left: {retries}")
            if retries == 0:
                print("Max retries reached. Skipping this sheet.")
                return None
        except Exception as e:
            print(f"Unexpected Error: {e}")
            return None


# Function to process all CSV files in a base folder
def analyze_csv_files(base_folder, llm):
    results = []

    for folder in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".csv"):
                    file_path = os.path.join(folder_path, file)
                    print(f"Processing file: {file_path}")

                    df = pd.read_csv(file_path)
                    response = analyze_sheet(dataframe=df, llm=llm)
                    if response:
                        # Unpack the details dictionary into individual columns
                        details = response.details
                        results.append(
                            {
                                'sheet_name': folder,
                                "file": file,
                                "is_inventory_planning": response.is_inventory_planning,
                                "description": response.description,
                                "SKU": details.get("SKU"),
                                "Location/Warehouse": details.get("Location/Warehouse"),
                                "Quantity": details.get("Quantity"),
                                "Total Inventory": details.get("Total Inventory"),
                                "Current Inventory": details.get("Current Inventory"),
                                "Sales Forecast": details.get("Sales Forecast"),
                            }
                        )
                    else:
                        print(f"Failed to analyze file: {file}")

    return pd.DataFrame(results)


# Main Execution
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
base_folder = "/Users/ajay/Documents/Atomic/inventory_analysis_2"
analysis_results_df = analyze_csv_files(base_folder, llm)

# Save results to a CSV file
output_file = "inventory_analysis_results.csv"
analysis_results_df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# Print the DataFrame
print(analysis_results_df)


Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/True Up Template/table_2.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/True Up Template/table_3.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/True Up Template/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_2.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_3.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_4.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_5.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_7.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/INVENTORY SNAPSHOT/table_6.csv
Processing file: /Users/aj

In [102]:
analysis_results_df

,sheet_name,file,is_inventory_planning,description,SKU,Location/Warehouse,Quantity,Total Inventory,Current Inventory,Sales Forecast
0,True Up Template,table_2.csv,0,The sheet does not contain identifiable column...,None,None,None,None,None,None
1,True Up Template,table_3.csv,1,This sheet contains data related to product or...,Unnamed: 10,Location,Cases,Unnamed: 12,Unnamed: 13,Unnamed: 14
2,True Up Template,table_1.csv,1,This sheet is used for inventory planning as i...,Unnamed: 3,Date,Unnamed: 4,Unnamed: 6,Unnamed: 5,Unnamed: 12
3,INVENTORY SNAPSHOT,table_2.csv,1,This sheet contains information about SKU desc...,Unnamed: 1,Unnamed: 6,Unnamed: 9,Unnamed: 8,Unnamed: 10,Unnamed: 18
4,INVENTORY SNAPSHOT,table_3.csv,1,This sheet appears to be used for inventory pl...,None,None,Unnamed: 5,Unnamed: 1,Unnamed: 6,None
...,...,...,...,...,...,...,...,...,...,...
185,Inventory - Monthly NEW,table_1.csv,1,This sheet contains data related to production...,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
186,INPUT -> DOT Inventory Report,table_2.csv,1,This sheet contains information about stocked ...,Unnamed: 1,Unnamed: 2,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
187,INPUT -> DOT Inventory Report,table_1.csv,1,This sheet is used for inventory planning as i...,DOT ITEM NUMBER,DOT DC,QTY ON HAND,OVERALL BALANCE,NET AVAILABLE,CASE WKLY FORECAST
188,Item List,table_1.csv,0,The sheet does not contain any identifiable co...,None,None,None,None,None,None


## Testing side

### Test Table Extraction

In [56]:
from openpyxl import load_workbook



In [60]:
len(sheet_names)

12

Cost  for sheet 1 : $1.44 (ran with gpt-4 )

In [59]:
import traceback
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook

# Load Excel Workbook
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 1 - Inventory Planning.xlsx'
workbook = load_workbook(file_path, data_only=False, read_only=True)
sheet_names = [sheet_name for sheet_name in workbook.sheetnames if not workbook[sheet_name].sheet_state == 'hidden']

# Function to extract a 50xN chunk from a sheet
def extract_sheet_chunk(sheet_name):
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    if sheet_name not in sheet_names:
        log_message = f"Sheet '{sheet_name}' doesn't exist. Available sheets: {sheet_names}"
        print(log_message)
        return log_message
    print(f"Fetching 50xN chunk from sheet: {sheet_name}")
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    data = [[cell.value for cell in row] for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols)]
    formatted_chunk = "\n".join([", ".join(map(str, row)) for row in data])
    print(f"Fetched chunk from sheet '{sheet_name}':\n{formatted_chunk[:500]}...")  # Show first 500 chars for brevity
    return formatted_chunk

# Function to analyze a chunk and generate code
import re

# def analyze_and_generate_code(sheet_name: str, chunk: str) -> str:
#     llm_input = f"""
#     You are a Python code generator for table extraction.
#     Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

#     {chunk}

#     Tasks:
#     1. Analyze the data to identify all tables present.
#     2. Generate Python code that reads data from the provided Excel file ('{file_path}') and the specified sheet ('{sheet_name}').
#     3. Create a new folder named '{sheet_name}' in the current working directory.
#     4. Extract each identified table and save it as a separate CSV file in the folder '{sheet_name}'.
#     """
#     print(f"Sending data to LLM for analysis and code generation...")
#     llm = ChatOpenAI(model="gpt-4", temperature=0)
#     try:
#         response = llm.invoke(llm_input)
#         generated_response = response.content  # Extract string content
#         print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

#         # Extract Python code block using regex
#         match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
#         if match:
#             generated_code = match.group(1).strip()  # Extract the Python code
#             print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
#             return generated_code
#         else:
#             print("No valid Python code block found in the LLM response.")
#             return "Error: No valid Python code block found."
#     except Exception as e:
#         print(f"Error during code generation: {e}")
#         return f"Error during code generation: {e}"

# best working - Only gets all the csvs. Doesn't get formulas yet
def analyze_and_generate_code(sheet_name: str, chunk: str) -> str:
    llm_input = f"""
    You are a Python code generator for table extraction.
    Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

    {chunk}

    Tasks:
    1. Analyze the data to identify all tables present in the chunk. Tables are defined as contiguous blocks of data separated by blank rows or rows with no data.
    2. Generate Python code that:
       - Reads data from the provided Excel file ('{file_path}') and specified sheet ('{sheet_name}').
       - Creates a new folder named '{sheet_name}'.
       - Extracts each identified table based on the above definition and saves it as a separate CSV file in the folder.
    3. Do not make any assumptions about the meaning or functionality of specific columns unless explicitly mentioned.
    """
    print(f"Sending data to LLM for analysis and code generation...")
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    try:
        response = llm.invoke(llm_input)
        generated_response = response.content  # Extract string content
        print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

        # Extract Python code block using regex
        match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
        if match:
            generated_code = match.group(1).strip()  # Extract the Python code
            print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
            return generated_code
        else:
            print("No valid Python code block found in the LLM response.")
            return "Error: No valid Python code block found."
    except Exception as e:
        print(f"Error during code generation: {e}")
        return f"Error during code generation: {e}"





# Function to execute generated Python code
def execute_code(code: str) -> str:
    try:
        print(f"Executing the following code:\n{code}...")  # Log the extracted code
        repl_tool = PythonREPLTool()
        result = repl_tool.run(code)
        print("Execution result:")
        print(result)
        return result
    except Exception as e:
        error_message = f"Execution Error: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        return error_message

# Main execution process
def run_analysis(sheet_name):
    print(f"Starting analysis for sheet: {sheet_name}")
    
    # Step 1: Fetch sheet chunk
    chunk = extract_sheet_chunk(sheet_name)
    if "doesn't exist" in chunk:
        print(f"Error: {chunk}")
        return chunk

    # Step 2: Analyze and generate code
    print("Analyzing the data and generating Python code...")
    code = analyze_and_generate_code(sheet_name, chunk)
    if "Error during code generation" in code:
        return code

    # Step 3: Attempt to execute code with retries
    retry = 10
    while retry > 0:
        result = execute_code(code)
        if "Execution Error" not in result:
            print("Code executed successfully!")
            return result  # Successful execution

        # Provide feedback and request fixes
        error_message = f"\n\nError Message: {result}"
        print(f"Retrying... ({10 - retry + 1}/10)")
        print(f"Requesting LLM to fix the code. Error: {error_message}")
        code = analyze_and_generate_code(sheet_name, chunk + error_message)
        retry -= 1

    print("Exceeded maximum retries. Could not execute code successfully.")
    return "Failed after multiple retries."

# Run the workflow
# sheet_to_analyze = "True Up Template"  # Replace with your target sheet name

for sheet_to_analyze in sheet_names:
    print('-'*40,f'\n {sheet_to_analyze}')
    print(run_analysis(sheet_to_analyze))


---------------------------------------- 
 INVENTORY SNAPSHOT
Starting analysis for sheet: INVENTORY SNAPSHOT
Fetching 50xN chunk from sheet: INVENTORY SNAPSHOT
Fetched chunk from sheet 'INVENTORY SNAPSHOT':
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
None, None, 2025-01-10 00:00:00, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 45 = antsy, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
None, None, None, None, None, None, None, None, N...
Analyzing the data and generating Python code...
Sending data to LLM for analysis and code generation...
Full LLM response:
Here is the Python code that accomplishes the tasks:

```python
import pandas as pd
import os

# Read data from the provided Excel file and specified sheet
df = pd.

In [64]:
for sheet_to_analyze in sheet_names[-4:]:
    print('-'*40,f'\n {sheet_to_analyze}')
    print(run_analysis(sheet_to_analyze))

---------------------------------------- 
 Delivery by Month
Starting analysis for sheet: Delivery by Month
Fetching 50xN chunk from sheet: Delivery by Month
Fetched chunk from sheet 'Delivery by Month':
None, None

None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, FOODSERVICE, RETAIL, None, None, None, None, None, None
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, =sum(L4,J4,B4,C4,D4,E4,F4,I4,N4), =sum(K4,H4,G4), None, None, None, None, None, None, None, None
None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, =sum(L5,J5,B5,C5,D5,E5,F5,I5,N5), =sum(K5,H5,G5), None, None, Non...
Analyzing the data and generating Python code...
Sending data to LLM for analysis and code generation...
Full LLM response:
Here is the Python code that accomplishes the tasks:

```python
import pandas as pd
import os

# Read the excel file
df = pd.read_excel('/Users/ajay/Documents/Atomic/in

This threw in the folders everywhere
Cost: 0.05 USD

In [1]:
import os
import traceback
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook

# Load Excel Workbook
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 2 - Supply Management.xlsx'
workbook_name = os.path.splitext(os.path.basename(file_path))[0]  # Get workbook name without extension
workbook = load_workbook(file_path, data_only=False, read_only=True)
sheet_names = [sheet_name for sheet_name in workbook.sheetnames if not workbook[sheet_name].sheet_state == 'hidden']

# Create a base directory with the workbook name
base_dir = os.path.join(os.getcwd(), workbook_name)
os.makedirs(base_dir, exist_ok=True)

# Function to extract a 50xN chunk from a sheet
def extract_sheet_chunk(sheet_name):
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    if sheet_name not in sheet_names:
        log_message = f"Sheet '{sheet_name}' doesn't exist. Available sheets: {sheet_names}"
        print(log_message)
        return log_message
    print(f"Fetching 50xN chunk from sheet: {sheet_name}")
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    data = [[cell.value for cell in row] for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols)]
    formatted_chunk = "\n".join([", ".join(map(str, row)) for row in data])
    print(f"Fetched chunk from sheet '{sheet_name}':\n{formatted_chunk[:500]}...")  # Show first 500 chars for brevity
    return formatted_chunk

# Function to analyze a chunk and generate code
import re

def analyze_and_generate_code(sheet_name: str, chunk: str) -> str:
    llm_input = f"""
    You are a Python code generator for table extraction.
    Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

    {chunk}

    Tasks:
    1. Analyze the data to identify all tables present in the chunk. Tables are defined as contiguous blocks of data separated by blank rows or rows with no data.
    2. Generate Python code that:
       - Reads data from the provided Excel file ('{file_path}') and specified sheet ('{sheet_name}').
       - Creates a new folder named '{sheet_name}' inside the folder '{workbook_name}' in the current working directory.
       - Extracts each identified table based on the above definition and saves it as a separate CSV file in the folder.
    3. Do not make any assumptions about the meaning or functionality of specific columns unless explicitly mentioned.
    """
    print(f"Sending data to LLM for analysis and code generation...")
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    try:
        response = llm.invoke(llm_input)
        generated_response = response.content  # Extract string content
        print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

        # Extract Python code block using regex
        match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
        if match:
            generated_code = match.group(1).strip()  # Extract the Python code
            print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
            return generated_code
        else:
            print("No valid Python code block found in the LLM response.")
            return "Error: No valid Python code block found."
    except Exception as e:
        print(f"Error during code generation: {e}")
        return f"Error during code generation: {e}"

# Function to execute generated Python code
def execute_code(code: str) -> str:
    try:
        print(f"Executing the following code:\n{code}...")  # Log the extracted code
        repl_tool = PythonREPLTool()
        result = repl_tool.run(code)
        print("Execution result:")
        print(result)
        return result
    except Exception as e:
        error_message = f"Execution Error: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        return error_message

# Main execution process
def run_analysis(sheet_name):
    print(f"Starting analysis for sheet: {sheet_name}")
    
    # Step 1: Fetch sheet chunk
    chunk = extract_sheet_chunk(sheet_name)
    if "doesn't exist" in chunk:
        print(f"Error: {chunk}")
        return chunk

    # Step 2: Analyze and generate code
    print("Analyzing the data and generating Python code...")
    code = analyze_and_generate_code(sheet_name, chunk)
    if "Error during code generation" in code:
        return code

    # Step 3: Attempt to execute code with retries
    retry = 10
    while retry > 0:
        result = execute_code(code)
        if "Execution Error" not in result:
            print("Code executed successfully!")
            return result  # Successful execution

        # Provide feedback and request fixes
        error_message = f"\n\nError Message: {result}"
        print(f"Retrying... ({10 - retry + 1}/10)")
        print(f"Requesting LLM to fix the code. Error: {error_message}")
        code = analyze_and_generate_code(sheet_name, chunk + error_message)
        retry -= 1

    print("Exceeded maximum retries. Could not execute code successfully.")
    return "Failed after multiple retries."

# Run the workflow for each sheet
for sheet_to_analyze in sheet_names:
    # Create a folder for each sheet under the workbook folder
    sheet_folder = os.path.join(base_dir, sheet_to_analyze)
    os.makedirs(sheet_folder, exist_ok=True)
    print('-'*40, f'\nAnalyzing Sheet: {sheet_to_analyze}')
    print(run_analysis(sheet_to_analyze))


---------------------------------------- 
Analyzing Sheet: SKU List
Starting analysis for sheet: SKU List
Fetching 50xN chunk from sheet: SKU List
Fetched chunk from sheet 'SKU List':
product, title, Cost, Price, Smart SKU id, Supplier, Menu, Category, Qty Per Case, Share
000110001508, Ground Beef 85/15, 15.0, None, =A2*1, Rastelli's, CORE, Beef, 18.0, =sumifs('Take rate per SKU'!DU8:DU89,'Take rate per SKU'!A8:A89,A2)
000110003502, Ground Beef 80/20, 15.0, None, 110003502.0, Mission Driven, CORE, Beef, 18.0, =sumifs('Take rate per SKU'!DU9:DU90,'Take rate per SKU'!A9:A90,A3)
000110001607, 100% Grass-Fed Ground Beef, 15.0, None, =A4*1, Rastelli's, CORE, Beef, 18.0, =sumifs('T...
Analyzing the data and generating Python code...
Sending data to LLM for analysis and code generation...


/var/folders/2j/n6fpg8f54xb5ld5v6bg1wgl80000gn/T/ipykernel_36661/2112633862.py:51: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
Python REPL can execute arbitrary code. Use with caution.


Full LLM response:
To accomplish the tasks outlined, we will write a Python script that uses the `pandas` library to read the Excel file, identify the tables, and save them as separate CSV files. Below is the code that performs these operations:

```python
import pandas as pd
import os

# Define the path to the Excel file and the sheet name
excel_file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 2 - Supply Management.xlsx'
sheet_name = 'SKU List'

# Read the data from the specified Excel sh...
Extracted code:
import pandas as pd
import os

# Define the path to the Excel file and the sheet name
excel_file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 2 - Supply Management.xlsx'
sheet_name = 'SKU List'

# Read the data from the specified Excel sheet
data = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Create a new folder named 'SKU List' inside the specified directory
output_folder = os.path.join(os.getcwd(), 'Company 2 - Supply M

/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68504 is marked as a date but the serial value 310001902.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68505 is marked as a date but the serial value 310002008.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68586 is marked as a date but the serial value 310001902.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68587 is marked as a date but the serial value 310002008.0 is outside the limits for dates

Execution result:
Extracted 1 tables and saved them as CSV files in '/Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/SKU depletions'.

Code executed successfully!
Extracted 1 tables and saved them as CSV files in '/Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/SKU depletions'.

---------------------------------------- 
Analyzing Sheet: Rollling Sales Forecast
Starting analysis for sheet: Rollling Sales Forecast
Fetching 50xN chunk from sheet: Rollling Sales Forecast
Fetched chunk from sheet 'Rollling Sales Forecast':
None, Wk0, Wk 1, Wk 2, Wk 3, Wk 4, Wk 5, Wk 6, Wk 7, Wk 8, Wk 9, Wk 10, Wk 11, Wk 12, Wk 13, Wk 14, Wk 15, Wk 16, Wk 17, Wk 18, Wk 19, Wk 20, Wk 21, Wk 22, Wk 23, Wk 24, Wk 25, Wk 26, Wk 27, Wk 28, Wk 29, Wk 30, Wk 31, Wk 32, Wk 33, Wk 34, Wk 35, Wk 36, Wk 37, Wk 38, Wk 39, Wk 40, Wk 41, Wk 42, Wk 43, Wk 44, Wk 45, Wk 46, Wk 47, Wk 48, Wk 49, Wk 50, Wk 51, Wk 52, Wk 53, Wk 54, Wk 55, Wk 56, Wk 57, Wk 58, Wk 5

/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68504 is marked as a date but the serial value 310001902.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68505 is marked as a date but the serial value 310002008.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68586 is marked as a date but the serial value 310001902.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/Users/ajay/miniconda3/envs/crustdata/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell C68587 is marked as a date but the serial value 310002008.0 is outside the limits for dates

Execution result:
Extracted 2 tables and saved them to '/Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/DC Fcst'

Code executed successfully!
Extracted 2 tables and saved them to '/Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/DC Fcst'

---------------------------------------- 
Analyzing Sheet: Sheet99
Starting analysis for sheet: Sheet99
Fetching 50xN chunk from sheet: Sheet99
Fetched chunk from sheet 'Sheet99':
5778.0, 6402.0, 6584.0, 6159.0, 6491.0, 6492.0, 6916.0, 5904.0, 5864.0, 6148.0, 6507.223323, 6401.169147999999, 6698.948286, 6729.51055, 6733.489131, 6685.658600999999, 7398.131401000001, 7516.267332000001, 7581.4740059999995, 7604.678448, 7965.889652, 8099.334318000001, 8193.715719, 8274.442824000002, 8532.236854, 8676.624462, 8780.585454, 8872.907608000001, 9080.907068000002, 9233.223610000001, 8021.093799, 8088.012507, 9510.996492000002, 10000.0, =AH1*1.01, =AI1*1.01, =AJ1*1.01, =AK1*1.01, =A...
Analyzing the 

Data is getting written into the right path
0.06 $

In [2]:
import os
import traceback
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from openpyxl import load_workbook

# Load Excel Workbook
file_path = '/Users/ajay/Documents/Atomic/inventory_analysis/Data/Company 3 - Inventory Dashboard _V2.xlsx'
workbook_name = os.path.splitext(os.path.basename(file_path))[0]  # Get workbook name without extension
workbook = load_workbook(file_path, data_only=False, read_only=True)
sheet_names = [sheet_name for sheet_name in workbook.sheetnames if not workbook[sheet_name].sheet_state == 'hidden']

# Create a base directory with the workbook name
base_dir = os.path.join(os.getcwd(), workbook_name)
os.makedirs(base_dir, exist_ok=True)

# Function to extract a 50xN chunk from a sheet
def extract_sheet_chunk(sheet_name):
    sheet_name = sheet_name.strip().replace('"', '').replace("'", "")
    if sheet_name not in sheet_names:
        log_message = f"Sheet '{sheet_name}' doesn't exist. Available sheets: {sheet_names}"
        print(log_message)
        return log_message
    print(f"Fetching 50xN chunk from sheet: {sheet_name}")
    sheet = workbook[sheet_name]
    max_cols = sheet.max_column
    data = [[cell.value for cell in row] for row in sheet.iter_rows(min_row=1, max_row=50, max_col=max_cols)]
    formatted_chunk = "\n".join([", ".join(map(str, row)) for row in data])
    print(f"Fetched chunk from sheet '{sheet_name}':\n{formatted_chunk[:500]}...")  # Show first 500 chars for brevity
    return formatted_chunk

# Function to analyze a chunk and generate code
import re

def analyze_and_generate_code(sheet_name: str, chunk: str, sheet_folder: str) -> str:
    llm_input = f"""
    You are a Python code generator for table extraction.
    Below is a 50xN chunk of data from the sheet '{sheet_name}' in an Excel file.

    {chunk}

    Tasks:
    1. Analyze the data to identify all tables present in the chunk. Tables are defined as contiguous blocks of data separated by blank rows or rows with no data.
    2. Generate Python code that:
       - Reads data from the provided Excel file ('{file_path}') and specified sheet ('{sheet_name}').
       - Saves all extracted tables as separate CSV files in the folder '{sheet_folder}'.
       - Ensures the generated code creates the CSV files in the exact folder specified and avoids creating additional folders elsewhere.
    3. Do not make any assumptions about the meaning or functionality of specific columns unless explicitly mentioned.
    """
    print(f"Sending data to LLM for analysis and code generation...")
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    try:
        response = llm.invoke(llm_input)
        generated_response = response.content  # Extract string content
        print(f"Full LLM response:\n{generated_response[:500]}...")  # Log full response

        # Extract Python code block using regex
        match = re.search(r"```python(.*?)```", generated_response, re.DOTALL)
        if match:
            generated_code = match.group(1).strip()  # Extract the Python code
            print(f"Extracted code:\n{generated_code[:500]}...")  # Log extracted code
            return generated_code
        else:
            print("No valid Python code block found in the LLM response.")
            return "Error: No valid Python code block found."
    except Exception as e:
        print(f"Error during code generation: {e}")
        return f"Error during code generation: {e}"

# Function to execute generated Python code
def execute_code(code: str) -> str:
    try:
        print(f"Executing the following code:\n{code}...")  # Log the extracted code
        repl_tool = PythonREPLTool()
        result = repl_tool.run(code)
        print("Execution result:")
        print(result)
        return result
    except Exception as e:
        error_message = f"Execution Error: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        return error_message

# Main execution process
def run_analysis(sheet_name):
    print(f"Starting analysis for sheet: {sheet_name}")
    
    # Step 1: Fetch sheet chunk
    chunk = extract_sheet_chunk(sheet_name)
    if "doesn't exist" in chunk:
        print(f"Error: {chunk}")
        return chunk

    # Step 2: Create the sheet folder
    sheet_folder = os.path.join(base_dir, sheet_name)
    os.makedirs(sheet_folder, exist_ok=True)

    # Step 3: Analyze and generate code
    print("Analyzing the data and generating Python code...")
    code = analyze_and_generate_code(sheet_name, chunk, sheet_folder)
    if "Error during code generation" in code:
        return code

    # Step 4: Attempt to execute code with retries
    retry = 10
    while retry > 0:
        result = execute_code(code)
        if "Execution Error" not in result:
            print("Code executed successfully!")
            return result  # Successful execution

        # Provide feedback and request fixes
        error_message = f"\n\nError Message: {result}"
        print(f"Retrying... ({10 - retry + 1}/10)")
        print(f"Requesting LLM to fix the code. Error: {error_message}")
        code = analyze_and_generate_code(sheet_name, chunk, sheet_folder)
        retry -= 1

    print("Exceeded maximum retries. Could not execute code successfully.")
    return "Failed after multiple retries."

# Run the workflow for each sheet
for sheet_to_analyze in sheet_names:
    print('-'*40, f'\nAnalyzing Sheet: {sheet_to_analyze}')
    print(run_analysis(sheet_to_analyze))


---------------------------------------- 
Analyzing Sheet: Product
Starting analysis for sheet: Product
Fetching 50xN chunk from sheet: Product
Fetched chunk from sheet 'Product':
SKU, Planning SKU, Category, Collection, Product Name, Priority, UPC, MSRP, COGS, Inbound, Ships By, Total On-Hand, Inventory Value, Supplier 1, Supplier 2, SKU MOQ , Collection MOQ , Lead Time, Billed Weight (lbs), Units Per MC, Weight (oz), weight lbs, next_gen_sku, HTS Codes, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
Bla-VerA-M08, None, Watches, Marble Auto, =iferror(VLOOKUP(A2,Sales!Z:AE,6,false),""), A, 749524585012.0, =...
Analyzing the data and generating Python code...
Sending data to LLM for analysis and code generation...
Full LLM response:
To accomplish the tasks you've outlined, we will create a Python script that uses the `pandas` library to read the specified Excel file and extract the tables from the 'Product' sheet. The scr

Cost: 0.02 USD

In [4]:
import os
import pandas as pd
from pydantic import BaseModel, Field, ValidationError
from langchain.chat_models import ChatOpenAI
import json


# Define the structured output schema
class InventorySheetAnalysis(BaseModel):
    is_inventory_planning: int = Field(
        ..., description="1 if the sheet is for inventory planning, 0 otherwise"
    )
    details: dict = Field(
        ...,
        description=(
            "A dictionary mapping columns to roles: SKU, Location/Warehouse, Quantity, "
            "Total Inventory, Current Inventory, Sales Forecast. Set to None if not found."
        ),
    )
    description: str = Field(
        ..., description="Explanation of why this sheet is for inventory planning."
    )


# Function to clean the LLM response
def clean_response(response: str) -> str:
    if response.startswith("```json"):
        response = response[7:]  # Remove ```json
    if response.endswith("```"):
        response = response[:-3]  # Remove ```
    return response.strip()  # Strip any extra whitespace


# Function to analyze a sheet using GPT-4o-mini
def analyze_sheet(dataframe, llm, retries=10):
    markdown_preview = dataframe.head(5).to_markdown()

    prompt = f"""
    You are a data analysis expert. Below is a preview of a sheet:

    {markdown_preview}

    Task:
    1. Determine if this sheet is used for inventory planning (1 for Yes, 0 for No).
    2. If Yes, provide:
       - 'details': A dictionary mapping column names to these roles: SKU, Location/Warehouse, Quantity, Total Inventory, Current Inventory, Sales Forecast. If no related column exists, set the value to None.
       - 'description': A brief explanation explaining why this sheet is for inventory planning.

    Respond strictly in the following JSON format:
    {{
        "is_inventory_planning": 1,
        "details": {{
            "SKU": "Column Name or None",
            "Location/Warehouse": "Column Name or None",
            "Quantity": "Column Name or None",
            "Total Inventory": "Column Name or None",
            "Current Inventory": "Column Name or None",
            "Sales Forecast": "Column Name or None"
        }},
        "description": "Brief explanation"
    }}
    """

    while retries > 0:
        try:
            response = llm.predict(prompt)
            cleaned_response = clean_response(response)
            structured_output = InventorySheetAnalysis.model_validate_json(cleaned_response)
            return structured_output
        except ValidationError as ve:
            print(f"Validation Error: {ve}")
            retries -= 1
            print(f"Retries left: {retries}")
            if retries == 0:
                print("Max retries reached. Skipping this sheet.")
                return None
        except Exception as e:
            print(f"Unexpected Error: {e}")
            return None


# Function to process all CSV files in a base folder
def analyze_csv_files(base_folder, llm):
    results = []

    for folder in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".csv"):
                    file_path = os.path.join(folder_path, file)
                    print(f"Processing file: {file_path}")

                    df = pd.read_csv(file_path)
                    response = analyze_sheet(dataframe=df, llm=llm)
                    if response:
                        # Unpack the details dictionary into individual columns
                        details = response.details
                        results.append(
                            {
                                'sheet_name': folder,
                                "file": file,
                                "is_inventory_planning": response.is_inventory_planning,
                                "description": response.description,
                                "SKU": details.get("SKU"),
                                "Location/Warehouse": details.get("Location/Warehouse"),
                                "Quantity": details.get("Quantity"),
                                "Total Inventory": details.get("Total Inventory"),
                                "Current Inventory": details.get("Current Inventory"),
                                "Sales Forecast": details.get("Sales Forecast"),
                            }
                        )
                    else:
                        print(f"Failed to analyze file: {file}")

    return pd.DataFrame(results)


# Main Execution
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
base_folders = ["/Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management", "/Users/ajay/Documents/Atomic/inventory_analysis_2/Company 3 - Inventory Dashboard _V2"]
for base_folder in base_folders:
    analysis_results_df = analyze_csv_files(base_folder, llm)

    # Save results to a CSV file
    output_file = f"{base_folder.split('/')[-1]}.csv"
    analysis_results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

    # Print the DataFrame
    # print(analysis_results_df)


Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/RLS/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Dry Ice Forecast/table_2.csv
Validation Error: 1 validation error for InventorySheetAnalysis
  Invalid JSON: expected ident at line 5 column 32 [type=json_invalid, input_value='{\n    "is_inventory_pla...ing and management."\n}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid
Retries left: 9
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Dry Ice Forecast/table_3.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Dry Ice Forecast/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Dry Ice Forecast/table_4.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/C

/var/folders/2j/n6fpg8f54xb5ld5v6bg1wgl80000gn/T/ipykernel_36661/1522898781.py:94: DtypeWarning: Columns (1,2,3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Rollling Sales Forecast/table_2.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Rollling Sales Forecast/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/NW/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Hf market SKU breakdown/table_2.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/Hf market SKU breakdown/table_1.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/CP Forecast (2023)/table_2.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/CP Forecast (2023)/table_3.csv
Processing file: /Users/ajay/Documents/Atomic/inventory_analysis_2/Company 2 - Supply Management/CP 